In [1]:
from utils import load_buffers,load_task_markers
import pandas as pd
import config
import os
import numpy as np

import mne
from mne import create_info
from mne.epochs import EpochsArray
from mne.decoding import Vectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt

import pickle
%matplotlib qt

In [2]:
# Loading files and forming the initial DataFrames
file_path = 'processed_data/motor_imagery/2024-08-10_14-25-31/task_data/task_markers.npz'  # Replace with your file path
df_tasks = load_task_markers(file_path)

folder_path = 'processed_data/motor_imagery/2024-08-10_14-25-31/signal_data/'
df_buffers = load_buffers(folder_path)

df_buffers = df_buffers.sort_values('timestamps').reset_index(drop=True)
df_tasks = df_tasks.sort_values('timestamps').reset_index(drop=True)

# Extracting data for the relevant channels
relevant_channels = 8  # For example, using 8 channels
df_buffers_raw = df_buffers.iloc[:, :relevant_channels] 

# Renaming the columns
channel_names = ['FP1', 'FP2', 'C3', 'C4', 'FC1', 'FC2', 'CP1', 'CP2']
df_buffers_raw.columns = channel_names

# Adding a column with timestamps
df_buffers_raw['timestamps'] = df_buffers['timestamps']

# Displaying the updated DataFrame
df_buffers_raw.head()


Array names in the .npz file: ['event_ids', 'timestamps']


,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
0,145946.625000,-187896.968750,18415.490234,30788.677734,16506.718750,37377.125000,31377.669922,38355.996094,2781.121965
1,145966.640625,-187873.781250,18383.996094,30775.000000,16480.968750,37434.144531,31533.416016,38574.394531,2781.126876
2,145922.609375,-187827.453125,18408.740234,30818.541016,16389.013672,37364.785156,31101.894531,38377.789062,2781.130896
3,145904.125000,-187824.437500,18434.511719,30837.025391,16410.181641,37343.304688,30938.279297,38250.765625,2781.134932
4,145907.046875,-187854.515625,18429.818359,30819.703125,16430.677734,37304.167969,31039.218750,38172.152344,2781.138932


In [3]:
df_markers = df_tasks[df_tasks['event_ids'].isin([1,2,3,4,99,100])]

markers_map = {
            1 :'left_start',
            2: 'left_end',
            3:'right_start',
            4:'right_end',
            99:'rest_start',
            100:'rest_end'
}

# Initialize lists for onsets, durations, and descriptions
onsets = []
durations = []
descriptions = []

# Create a dictionary to keep track of start times
start_times = {}

# Iterate over the DataFrame rows
for index, row in df_markers.iterrows():
    event_id = row['event_ids']
    timestamp = row['timestamps']

    if event_id in markers_map:
        description = markers_map[event_id]

        # Check if it is a start event
        if 'start' in description:
            # Store the start time
            start_times[description.split('_')[0]] = timestamp
        elif 'end' in description:
            # Determine the corresponding start event type
            event_type = description.split('_')[0]  # e.g., 'left', 'right', 'rest'
            
            if event_type in start_times:
                start_time = start_times[event_type]
                duration = timestamp - start_time
                
                # Append to lists
                onsets.append(start_time)
                durations.append(duration)
                descriptions.append(event_type.upper())
                # Remove the start time to avoid re-use
                del start_times[event_type]

# Print the results
print("Onsets:", onsets)
print("Durations:", durations)
print("Descriptions:", descriptions)

Onsets: [2781.3794196, 2784.4033565, 2785.4367057, 2788.4701555, 2789.5035212, 2792.5371476, 2793.5704891, 2796.6038579, 2797.637192, 2800.6705171, 2801.7038715, 2804.737221]
Durations: [3.007221500000014, 1.0174547999999959, 3.0173180000001594, 1.017410899999959, 3.0174406000001, 1.0176989000001413, 3.0179434000001493, 1.0177365999998074, 3.0175421999997525, 1.017865900000288, 3.0177842000002784, 1.0176347999999962]
Descriptions: ['LEFT', 'REST', 'RIGHT', 'REST', 'LEFT', 'REST', 'RIGHT', 'REST', 'LEFT', 'REST', 'RIGHT', 'REST']


In [5]:
# Creating separate DataFrames for each event
left_events = []
right_events = []
rest_events = []
# Iterating through onsets, durations, and descriptions, creating a DataFrame for each event
for onset, duration, description in zip(onsets, durations, descriptions):
    event_df = df_buffers_raw[(df_buffers_raw['timestamps'] >= onset) & (df_buffers_raw['timestamps'] <= (onset + duration))]
    
    if description == 'LEFT':
        left_events.append(event_df)
    elif description == 'RIGHT':
        right_events.append(event_df)
    elif description == 'REST':
        rest_events.append(event_df)

# Displaying the first few DataFrames for verification
for i, df in enumerate(left_events[:3]):
    print(f"Left Event {i+1} DataFrame:")
    display(df)

for i, df in enumerate(right_events[:3]):
    print(f"Right Event {i+1} DataFrame:")
    display(df)
    # spectrum = df.compute_psd()

# for i, df in enumerate(rest_events[:3]):
#     print(f"Rest Event {i+1} DataFrame:")
#     display(df)


Left Event 1 DataFrame:


,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
65,145958.031250,-187910.125000,18415.400391,30791.628906,16521.894531,37371.960938,31337.548828,38331.296875,2781.382939
66,145999.781250,-187878.609375,18383.818359,30775.066406,16517.894531,37450.636719,31553.265625,38595.203125,2781.386976
67,145970.453125,-187826.515625,18413.679688,30822.318359,16422.251953,37385.839844,31149.546875,38419.429688,2781.390985
68,145931.437500,-187835.140625,18445.396484,30849.789062,16440.802734,37362.773438,30944.314453,38274.367188,2781.394999
69,145922.375000,-187883.281250,18434.578125,30828.933594,16456.494141,37319.164062,31025.025391,38190.324219,2781.399001
...,...,...,...,...,...,...,...,...,...
813,146029.531250,-187662.828125,18439.294922,30857.857422,16468.408203,37436.871094,31184.908203,38444.398438,2784.370724
814,146007.656250,-187650.531250,18475.974609,30888.300781,16489.976562,37415.679688,30980.255859,38303.269531,2784.373768
815,146005.343750,-187695.156250,18468.666016,30870.017578,16506.091797,37372.699219,31057.949219,38212.632812,2784.376773
816,146028.843750,-187743.484375,18437.126953,30834.878906,16563.044922,37420.597656,31356.279297,38354.699219,2784.381716


Left Event 2 DataFrame:


,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
2099,146091.093750,-187709.953125,18392.556641,30760.203125,16499.320312,37435.886719,31464.460938,38577.457031,2789.507343
2100,146055.312500,-187646.421875,18434.490234,30813.400391,16430.722656,37375.222656,31027.193359,38355.773438,2789.511382
2101,146043.703125,-187654.859375,18456.259766,30826.542969,16458.998047,37355.062500,30919.191406,38244.816406,2789.514398
2102,146048.250000,-187708.562500,18445.308594,30799.921875,16478.175781,37315.542969,31041.500000,38179.863281,2789.518360
2103,146080.546875,-187731.968750,18421.949219,30770.820312,16552.830078,37408.730469,31398.970703,38417.328125,2789.522402
...,...,...,...,...,...,...,...,...,...
2849,146018.015625,-187775.281250,18368.753906,30740.755859,16532.625000,37393.886719,31396.914062,38405.953125,2792.504332
2850,146031.593750,-187737.515625,18357.218750,30743.304688,16478.107422,37415.167969,31404.625000,38526.292969,2792.508353
2851,145992.500000,-187693.359375,18391.171875,30787.113281,16412.685547,37350.613281,30969.929688,38306.242188,2792.512375
2852,145981.609375,-187702.750000,18407.914062,30796.390625,16436.445312,37322.046875,30887.787109,38187.507812,2792.517305


Left Event 3 DataFrame:


...

,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
1081,145855.375000,-187889.171875,18368.082031,30737.894531,16438.166016,37271.265625,31123.552734,38166.519531,2785.440445
1082,145897.812500,-187887.312500,18338.443359,30707.966797,16490.736328,37381.750000,31485.203125,38478.191406,2785.444445
1083,145885.343750,-187842.562500,18340.455078,30728.216797,16394.244141,37346.925781,31272.861328,38443.703125,2785.448506
1084,145846.875000,-187818.843750,18380.867188,30772.562500,16371.356445,37301.328125,30907.277344,38247.121094,2785.452489
1085,145843.968750,-187836.796875,18391.417969,30773.568359,16399.027344,37272.562500,30916.105469,38155.007812,2785.457445
...,...,...,...,...,...,...,...,...,...
1831,145992.781250,-187681.390625,18448.257812,30813.130859,16460.271484,37327.210938,30941.744141,38194.394531,2788.435860
1832,146002.218750,-187726.734375,18432.902344,30785.884766,16498.134766,37323.726562,31144.876953,38207.984375,2788.440801
1833,146043.765625,-187738.250000,18398.458984,30750.210938,16554.998047,37432.464844,31505.185547,38517.710938,2788.444848
1834,146039.187500,-187686.765625,18408.025391,30780.833984,16467.378906,37407.500000,31312.648438,38491.582031,2788.447874


Right Event 2 DataFrame:


,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
3116,146021.218750,-187643.437500,18383.550781,30780.876953,16419.726562,37362.101562,31081.933594,38382.550781,2793.571905
3117,145991.765625,-187637.984375,18413.099609,30807.050781,16434.208984,37334.476562,30870.732422,38229.843750,2793.574913
3118,145984.500000,-187682.375000,18407.623047,30787.761719,16452.179688,37291.203125,30950.572266,38141.753906,2793.578869
3119,146006.875000,-187723.890625,18380.978516,30750.679688,16512.306641,37343.976562,31255.472656,38299.132812,2793.581894
3120,146039.015625,-187714.703125,18349.642578,30731.927734,16511.501953,37423.234375,31481.984375,38561.296875,2793.585871
...,...,...,...,...,...,...,...,...,...
3867,146091.921875,-187500.046875,18432.656250,30822.853516,16477.125000,37412.863281,31131.242188,38447.636719,2796.569274
3868,146065.046875,-187500.046875,18461.601562,30846.591797,16487.138672,37385.214844,30914.898438,38293.812500,2796.574219
3869,146063.203125,-187511.281250,18457.490234,30834.232422,16502.248047,37341.562500,30985.531250,38196.828125,2796.578235
3870,146084.234375,-187554.406250,18429.662109,30800.771484,16559.849609,37390.847656,31280.082031,38340.214844,2796.582254


Right Event 3 DataFrame:


,FP1,FP2,C3,C4,FC1,FC2,CP1,CP2,timestamps
5153,146178.859375,-187500.046875,18390.634766,30770.417969,16549.767578,37472.945312,31431.693359,38615.210938,2801.707449
5154,146140.781250,-187500.046875,18433.394531,30820.597656,16491.720703,37412.371094,31000.259766,38391.175781,2801.710496
5155,146126.937500,-187500.046875,18451.945312,30828.554688,16518.117188,37384.656250,30922.341797,38275.863281,2801.714510
5156,146131.406250,-187500.046875,18437.238281,30799.228516,16539.531250,37351.886719,31060.320312,38226.691406,2801.719445
5157,146166.281250,-187500.046875,18402.728516,30762.302734,16604.328125,37445.140625,31419.289062,38482.441406,2801.723464
...,...,...,...,...,...,...,...,...,...
5903,146137.406250,-187500.046875,18407.957031,30772.853516,16608.419922,37455.957031,31399.640625,38470.460938,2804.702853
5904,146159.328125,-187500.046875,18392.199219,30770.148438,16569.974609,37498.269531,31475.681641,38640.308594,2804.707779
5905,146123.687500,-187500.046875,18433.640625,30822.585938,16500.416016,37435.460938,31045.298828,38421.152344,2804.711822
5906,146106.609375,-187500.046875,18453.265625,30835.751953,16524.064453,37412.839844,30935.105469,38306.464844,2804.714835
